In [121]:
from utils import *
from pprint import pprint

In [122]:
house = importfile('hw3_house_votes_84.csv', ',')
housecategory = {}
for i in house[0]:
    housecategory[i] = 'categorical'
housecategory["class"] = 'class'
housedata = np.array(house[1:]).astype(float)

# print(len(housedata))
# print(housecategory)

#class in last column

In [123]:
wine = importfile('hw3_wine.csv', '\t')
winecategory = {}
for i in wine[0]:
    winecategory[i] = 'numerical'
winecategory["# class"] = 'class'
winedata = np.array(wine[1:]).astype(float)
# winedata = np.array([[float(item) for item in onelist] for onelist in wine[1:]])

# print(len(winedata))
# print(winecategory)

#class in first column

In [124]:
cancer = importfile('hw3_cancer.csv', '\t')
cancercategory = {}
for i in cancer[0]:
    cancercategory[i] = 'numerical'
cancercategory["Class"] = 'class'
cancerdata = np.array(cancer[1:]).astype(float)

#print(len(cancerdata))
#print(cancercategory)

#class in last column

In [125]:
cmc = importfile('cmc.data', ',')
cmccategory = {"Wife's age":"numerical","Wife's education":"categorical",
"Husband's education":"categorical","Number of children ever born":"numerical",
"Wife's religion":"binary","Wife's now working?":"binary",
"Husband's occupation":"categorical","Standard-of-living index":"categorical",
"Media exposure":"binary","Contraceptive method used":"class"}
cmcdata = np.array(cmc).astype(int)

# print(len(cmcdata))
# print(cmccategory)

#class in last column

In [126]:
w = {list(cmccategory.keys())[0]:cmccategory[list(cmccategory.keys())[0]]}

print(w)

{"Wife's age": 'numerical'}


In [127]:
def id3cat(collist, listattribution):
    original_ent = entropy(collist[-1])
    smallest_ent = 1
    i = 0

    # bestindex = i
    best = listattribution[i]
    for attributes in listattribution[:-1]: # I keep the last column: the target/label.
        liskey = list(Counter(collist[i]).keys())
        listofcategory = []
        for value in liskey:
            index = [idx for idx, element in enumerate(collist[i]) if element == value]
            category = np.array(collist[-1][index]) 
            listofcategory.append(category) # list of nparrays of target/label/categories.

        ent = 0
        for cat in listofcategory:
            a = len(cat)/len(collist[i]) # This is probability
            ent += a * entropy(cat) # probability multiple by entropy

        if ent < smallest_ent:
            smallest_ent = ent
            best = attributes
            # bestindex = i
        i+=1

    return best, original_ent-smallest_ent

In [128]:
def id3bestseperate(dataset, attributes:dict):
    # dataset in is the dataset by row. 
    # attributes is the dictionary of attributes:type 
    # types: numerical, categorical, binary.
    datasetbycolumn = dataset.T
    classindex = list(attributes.values()).index("class")
    originalentrophy = entropy(datasetbycolumn[classindex])
    smallestentrophy = 1

    thresholdvalue = -1

    gain = originalentrophy-smallestentrophy
    i = 0
    bestattribute = {list(attributes.keys())[i]:attributes[list(attributes.keys())[i]]}
    attributesinuse = list(attributes.keys())[1:] if (classindex == 0) else list(attributes.keys())[:classindex]
    datasetinuse = datasetbycolumn[1:] if (classindex == 0) else datasetbycolumn[:classindex]

    for attribute in attributesinuse:
        if attributes[attribute] == "categorical" or attributes[attribute] == "binary":
            listofkeys = list(Counter(datasetinuse[i]).keys())
            listofcategory = [] # this is the list of categorical values.
            
            for key in listofkeys:
                indexlist = [idx for idx, element in enumerate(datasetinuse[i]) if element == key]
                category = np.array(datasetbycolumn[classindex][indexlist])
                listofcategory.append(category)

            entropynow = 0

            for ctgry in listofcategory:
                a = len(ctgry)/len(datasetinuse[i]) # This is probability
                entropynow += a * entropy(ctgry)

            if entropynow < smallestentrophy:
                smallestentrophy = entropynow
                bestattribute = {attribute:attributes[attribute]}
                
        elif attributes[attribute] == "numerical":
            k="ahah"
            
        i += 1
    # set first attribution dictionary {key:type} to the best attributes.
    return bestattribute, thresholdvalue, gain

In [ ]:
housedata,housecategory

In [129]:
bestattribute, thresholdvalue, gain = id3bestseperate(housedata,housecategory)
print(bestattribute)

{'physician-fee-freeze': 'categorical'}


In [130]:
ba,ga = id3cat(housedata.T,list(housecategory.keys()))
ba

'physician-fee-freeze'